# 모델 매개변수 최적화
- 모델을 학습하는 과정은 반복적인 과정을 거침
- 에폭(epoch)이라고 부르는 각 반복 단계에서 모델은 출력을 추측
- 추측과 정답 사이의 손실을 계산
- 매개변수에 대한 오류의 도함수(derivative)를 수집
- 경사하강법을 사용하여 이 파라매터들을 최적화(optimize)

In [2]:
import torch 
from torch import nn 
from torch.utils.data import DataLoader 
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root = 'data',
    train = True, 
    download=True,
    transform=ToTensor() 
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

In [3]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits 

model = NeuralNetwork()

# HyperParameter 
- 모델 최적화 과정을 제어할 수 있는 매개변수 
- 모델 학습과 convergence rate에 영향을 미침 

## 학습시 조정하는 하이퍼파라미터
- epoch 수 : 데이터셋을 반복하는 횟수
- batch size : 매개변수가 갱신되기 전 신경망을 통해 전파된 데이터 샘플 수 
- learning rate : 각 배치/에폭에서 모델의 매개변수를 조정하는 비율 

In [5]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

# Loss Function 
- 학습되지 않은 신경망은 정확도가 높지 않을 확률이 높음 
- loss function은 결과값과 실제 값 사이의 차이를 측정하여 학습 중에 이 값을 최소화 하려고 함 
- nn.MSELoss : regression task (평균 제곱 오차)
- nn.CrossEntropyLoss : classification

In [6]:
loss_fn = nn.CrossEntropyLoss()

# Optimization Loop
- 하이퍼 파라미터를 설정한 뒤 최적화 단계를 통해 모델을 학습 
- 최적화 단계의 각 반복을 에폭이라 함 
- 하나의 에폭 
    - train loop : 학습용 데이터셋을 반복하고 최적의 매개변수로 수렴 
    - valid/test loop : 모델 성능이 개선되고 있는지를 확인하기 위해 테스트 데이터셋 반복

# Optimizer
- 최적화는 각 학습 단계에서 모델의 오류를 줄이기 위해 매개변수를 조정하는 과정 
- 최적화 알고리즘은 확률적 경사하강법 사용 

In [7]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) 

학습 단계(loop)에서 최적화는 세단계로 이뤄집니다:
- optimizer.zero_grad()를 호출하여 모델 매개변수의 변화도를 재설정
- 기본적으로 변화도는 더해지기(add up) 때문에 중복 계산을 막기 위해 반복할 때마다 명시적으로 0으로 설정
- loss.backwards()를 호출하여 예측 손실(prediction loss)을 역전파
- PyTorch는 각 매개변수에 대한 손실의 변화도를 저장
- 변화도를 계산한 뒤에는 optimizer.step()을 호출
- 역전파 단계에서 수집된 변화도로 매개변수를 조정

# 구현

In [8]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 예측(prediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.308284 [    0/60000]
loss: 2.298908 [ 6400/60000]
loss: 2.270551 [12800/60000]
loss: 2.266565 [19200/60000]
loss: 2.261388 [25600/60000]
loss: 2.228342 [32000/60000]
loss: 2.242568 [38400/60000]
loss: 2.206381 [44800/60000]
loss: 2.196143 [51200/60000]
loss: 2.171124 [57600/60000]
Test Error: 
 Accuracy: 38.1%, Avg loss: 2.164856 

Epoch 2
-------------------------------
loss: 2.176045 [    0/60000]
loss: 2.172585 [ 6400/60000]
loss: 2.107558 [12800/60000]
loss: 2.124751 [19200/60000]
loss: 2.092357 [25600/60000]
loss: 2.021230 [32000/60000]
loss: 2.064869 [38400/60000]
loss: 1.982366 [44800/60000]
loss: 1.976201 [51200/60000]
loss: 1.922351 [57600/60000]
Test Error: 
 Accuracy: 55.6%, Avg loss: 1.916674 

Epoch 3
-------------------------------
loss: 1.949857 [    0/60000]
loss: 1.931011 [ 6400/60000]
loss: 1.809937 [12800/60000]
loss: 1.851130 [19200/60000]
loss: 1.762668 [25600/60000]
loss: 1.700354 [32000/60000]
loss: 1.740554 [38400/